# Skip notebook test
(this notebook is not executed as part of the RAPIDS cuGraph CI process.  Execution will take a few hours)

---

# Release Benchmarking

With every release, RAPIDS publishes a release slide deck that includes the current performance state of cuGraph. 
This notebook, starting with release 0.15, runs all the various algorithms to computes the performance gain.  





    

### Timing 
When looking at the overall workflow, NetworkX and cuGraph do things differently.  For example, NetworkX spends a lot of time creating the graph data structure.  cuGraph on the other hand does a lazy creation of the data structure when an algorithm is called.  

To further complicate the comparison problem, NetworkX does not always return the answer.  In some cases it returns a generator that is then called to get the data.  

This benchmark will measure time from an analyst perspective, how long does it take to create the graph and run an algorithm.  

__What is not timed__:  Reading the data</p>
__What is timed__:     (1) creating a Graph, (2) running the algorithm (3) run any generators


Notes:
* Since this is clean test data, we do not need to renumber the data.
* use default arguments in most cases

---

### Algorithms
|        Algorithm        |  Type         | Graph | DiGraph |   Notes
| ------------------------|---------------|------ | ------- |-------------
| Katz                    | Centrality    |   X   |         | 
| Betweenness Centrality  | Centrality    |   X   |         | Estimated, k = 100
| Louvain                 | Community     |   X   |         | Uses python-louvain for comparison
| Triangle Counting       | Community     |   X   |         |
| WCC                     | Components    |       |    X    | Nx requires directed and returns a generator  
| Core Number             | Core          |   X   |         |  
| PageRank                | Link Analysis |       |    X    |
| Jaccard                 | Similarity    |   X   |         |
| BFS                     | Traversal     |   X   |         | No depth limit 
| SSSP                    | Traversal     |   X   |         | 


### Test Data
Users must run the _dataPrep.sh_ script before running this notebook so that the test files are downloaded

| File Name              | Num of Vertices | Num of Edges |
| ---------------------- | --------------: | -----------: |
| preferentialAttachment |         100,000 |      999,970 |
| dblp-2010              |         326,186 |    1,615,400 |
| coPapersCiteseer       |         434,102 |   32,073,440 |
| as-Skitter             |       1,696,415 |   22,190,596 |



### Notes
* Running Betweenness Centrality on the full graph is prohibited using NetworkX.  Anything over k=100 can explode runtime to days


Notebook Credits

    
| Author        |    Date    |  Update             | cuGraph Version |  Test Hardware         |
| --------------|------------|---------------------|-----------------|------------------------|
| Brad Rees     | 10/06/2020 | created             | 0.16            | GV100, CUDA 10.2       |
| Brad Rees     | 01/20/2022 | updated             | 22.02           | Quadro A6000 CUDA 11.5 |
| Brad Rees     | 01/20/2022 | added perf w/Nx obj | 22.02           | Quadro A6000 CUDA 11.5 |





## Import Modules

In [1]:
# system and other
import gc
import os
import time
import numpy as np
import math

# rapids
import cugraph
import cudf

# NetworkX libraries
import networkx as nx

# MTX file reader
from scipy.io import mmread

In [2]:
try: 
    import community
except ModuleNotFoundError:
    os.system('pip install python-louvain')
    import community

### Define the test data

In [3]:
# Test Files
# set the data argument for full test or quick test

data_full = {
    'preferentialAttachment' : './data/preferentialAttachment.mtx',
    'dblp'                   : './data/dblp-2010.mtx',
    'coPapersCiteseer'       : './data/coPapersCiteseer.mtx',
    'as-Skitter'             : './data/as-Skitter.mtx'
}

# for quick testing
data_quick = {
    'preferentialAttachment' : './data/preferentialAttachment.mtx',    
    #'karate' : './data/karate.mtx',
}



data = data_full


In [4]:
# Get the data - will auto skip if files exists
!./dataPrep.sh

### Read data
The data is read in once and used for both cuGraph and NetworkX.

In [5]:
# Data reader - the file format is MTX, so we will use the reader from SciPy
def read_data(datafile):
    print('Reading ' + str(datafile) + '...')
    M = mmread(datafile).asfptype()

    _gdf = cudf.DataFrame()
    _gdf['src'] = M.row
    _gdf['dst'] = M.col
    
    return _gdf

## Create Graph functions
There are two types of graphs created:
* Directed Graphs - calls to create_xx_digraph
* Undirected Graphs - calls to create_xx_ugraph <- fully syemmeterized 

In [6]:
# NetworkX
def create_nx_digraph(_df):
    _gnx = nx.from_pandas_edgelist(_df, source='src', target='dst', edge_attr=None, create_using=nx.DiGraph)
    return _gnx

def create_nx_ugraph(_df):
    _gnx = nx.from_pandas_edgelist(_df, source='src', target='dst', edge_attr=None, create_using=nx.Graph)
    return _gnx


# cuGraph
def create_cu_digraph(_df):
    _g = cugraph.Graph(directed=True)
    _g.from_cudf_edgelist(_df, source='src', destination='dst', renumber=False)
    return _g

def create_cu_ugraph(_df):
    _g = cugraph.Graph(directed=False)
    _g.from_cudf_edgelist(_df, source='src', destination='dst', renumber=False)
    return _g

## Algorithm Execution

### Katz

In [7]:
def nx_katz(_df, alpha):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    _ = nx.katz_centrality(_G, alpha)
    t2 = time.time() - t1
    return t2

def cu_katz(_df, alpha):
    t1 = time.time()
    _G = create_cu_ugraph(_df)
    _ = cugraph.katz_centrality(_G, alpha)
    t2 = time.time() - t1
    return t2

def cu_katz_nx(_df, alpha):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    _ = cugraph.katz_centrality(_G, alpha)
    t2 = time.time() - t1
    return t2

### Betweenness Centrality

In [8]:
def nx_bc(_df, _k):
    print(f" k = {_k}", end=' ')
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    _ = nx.betweenness_centrality(_G, k=_k)
    t2 = time.time() - t1
    return t2

def cu_bc(_df, _k):
    t1 = time.time()
    _G = create_cu_ugraph(_df)
    _ = cugraph.betweenness_centrality(_G, k=_k)
    t2 = time.time() - t1
    return t2

def cu_bc_nx(_df, _k):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    _ = cugraph.betweenness_centrality(_G, k=_k)
    t2 = time.time() - t1
    return t2

### Louvain

In [9]:
def nx_louvain(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    parts = community.best_partition(_G)
    
    # Calculating modularity scores for comparison 
    _ = community.modularity(parts, _G)  
    
    t2 = time.time() - t1
    return t2

def cu_louvain(_df):
    t1 = time.time()
    _G = create_cu_ugraph(_df)
    _,_ = cugraph.louvain(_G)
    t2 = time.time() - t1
    return t2

def cu_louvain_nx(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    _,_ = cugraph.louvain(_G)
    t2 = time.time() - t1
    return t2

### Triangle Counting

In [10]:
def nx_tc(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    nx_count = nx.triangles(_G)
    
    # To get the number of triangles, we would need to loop through the array and add up each count
    count = 0
    for key, value in nx_count.items():
        count = count + value    
    
    t2 = time.time() - t1
    return t2

def cu_tc(_df):
    t1 = time.time()
    _G = create_cu_ugraph(_df)
    _ = cugraph.triangles(_G)
    t2 = time.time() - t1
    return t2

def cu_tc_nx(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    _ = cugraph.triangles(_G)
    t2 = time.time() - t1
    return t2

### WCC

In [11]:
def nx_wcc(_df):
    t1 = time.time()
    _G = create_nx_digraph(_df)
    gen = nx.weakly_connected_components(_G)

    list_of_digraphs = []

    for subgraph in gen:
        list_of_digraphs.append(nx.subgraph(_G, subgraph))
    
    t2 = time.time() - t1
    return t2

def cu_wcc(_df):
    t1 = time.time()
    _G = create_cu_digraph(_df)    
    _ = cugraph.weakly_connected_components(_G)
    t2 = time.time() - t1
    return t2

def cu_wcc_nx(_df):
    t1 = time.time()
    _G = create_nx_digraph(_df)    
    _ = cugraph.weakly_connected_components(_G)
    t2 = time.time() - t1
    return t2

### Core Number

In [12]:
def nx_core_num(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    nx_count = nx.core_number(_G)
    
    count = 0
    for key, value in nx_count.items():
        count = count + value       
    
    t2 = time.time() - t1
    return t2

def cu_core_num(_df):
    t1 = time.time()
    _G = create_cu_ugraph(_df)    
    _ = cugraph.core_number(_G)
    t2 = time.time() - t1
    return t2

def cu_core_num_nx(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)    
    _ = cugraph.core_number(_G)
    t2 = time.time() - t1
    return t2

### PageRank

In [13]:
def nx_pagerank(_df):
    t1 = time.time()
    _G = create_nx_digraph(_df)
    _ = nx.pagerank(_G)
    t2 = time.time() - t1
    return t2

def cu_pagerank(_df):
    t1 = time.time()
    _G = create_cu_digraph(_df)
    _ = cugraph.pagerank(_G)
    t2 = time.time() - t1
    return t2

def cu_pagerank_nx(_df):
    t1 = time.time()
    _G = create_nx_digraph(_df)
    _ = cugraph.pagerank(_G)
    t2 = time.time() - t1
    return t2

### Jaccard

In [14]:
def nx_jaccard(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    _ = nx.jaccard_coefficient(_G)
    t2 = time.time() - t1
    return t2

def cu_jaccard(_df):
    t1 = time.time()
    _G = create_cu_ugraph(_df)
    _ = cugraph.jaccard_coefficient(_G)
    t2 = time.time() - t1
    return t2

def cu_jaccard_nx(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    _ = cugraph.jaccard_coefficient(_G)
    t2 = time.time() - t1
    return t2

### BFS

In [15]:
def nx_bfs(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    _ = nx.bfs_edges(_G, 1)
    t2 = time.time() - t1
    return t2

def cu_bfs(_df):
    t1 = time.time()
    _G = create_cu_ugraph(_df)
    _ = cugraph.bfs(_G, 1)
    t2 = time.time() - t1
    return t2

def cu_bfs_nx(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    _ = cugraph.bfs(_G, 1)
    t2 = time.time() - t1
    return t2

### SSSP

In [16]:
def nx_sssp(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)
    _ = nx.shortest_path(_G, 1)
    t2 = time.time() - t1
    return t2

def cu_sssp(_df):
    t1 = time.time()
    _G = create_cu_ugraph(_df)    
    _ = cugraph.sssp(_G, 1)
    t2 = time.time() - t1
    return t2

def cu_sssp_nx(_df):
    t1 = time.time()
    _G = create_nx_ugraph(_df)    
    _ = cugraph.sssp(_G, 1)
    t2 = time.time() - t1
    return t2

---

# Benchmark

In [17]:
# number of datasets
num_datasets = len(data)

In [18]:
# do a simple pass just to get all the libraries initiallized
# This cell might not be needed
v = './data/preferentialAttachment.mtx'
gdf = read_data(v)
pdf = gdf.to_pandas()
print(f"\tGDF Size {len(gdf)}")

g = create_cu_ugraph(gdf)

print(f"\tcugraph Size {g.number_of_edges()}")
print(f"\tcugraph Order {g.number_of_vertices()}")

gnx = create_nx_ugraph(pdf)

# clean up what we just created
del gdf
del pdf
del g
del gnx
gc.collect()

Reading ./data/preferentialAttachment.mtx...
	GDF Size 999970
	cugraph Size 499985
	cugraph Order 100000


0

In [19]:
# arrays to capture performance gains
names = []
algos = []

# Two dimension data [file, perf]
time_algo_nx = []          # NetworkX
time_algo_cu = []          # cuGraph
time_algo_cx = []          # cuGraph
perf = []
perf_cu_nx = []

algos.append("   ")

i = 0
for k,v in data.items():
    time_algo_nx.append([])
    time_algo_cu.append([])
    time_algo_cx.append([])
    perf.append([])
    perf_cu_nx.append([])
    
    # Saved the file Name
    names.append(k)

    # read data
    gdf = read_data(v)
    pdf = gdf.to_pandas()
    print(f"\tdata in gdf {len(gdf)} and data in pandas {len(pdf)}")

    # prep
    tmp_g = create_cu_ugraph(gdf)
    deg = tmp_g.degree()
    deg_max = deg['degree'].max()

    alpha = 1 / deg_max
    num_nodes = tmp_g.number_of_vertices()
    
    del tmp_g
    del deg
    
    
    #----- Algorithm order is same as defined at top ----
    
    #-- Katz 
    print("\tKatz  ", end = '')
    if i == 0: 
        algos.append("Katz")

    print("n.", end='')
    tx = nx_katz(pdf, alpha)
    print("c.", end='')
    tc = cu_katz(gdf, alpha)
    print("cx.", end='')
    tcx = cu_katz_nx(pdf, alpha)
    print("")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    time_algo_cx[i].append(tcx)
    perf[i].append(tx/tc)
    perf_cu_nx[i].append(tx/tcx)
    gc.collect()
    
    
    #-- BC
    print("\tBC k=100  ", end='')
    if i == 0:
        algos.append("BC Estimate fixed")

    k = 100
    if k > num_nodes:
        k = num_nodes
        
    print("n.", end='')
    tx = nx_bc(pdf, k)
    print("c.", end='')
    tc = cu_bc(gdf, k)
    print("cx.", end='')
    tcx = cu_bc_nx(pdf, k)
    print(" ")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    time_algo_cx[i].append(tcx)
    perf[i].append(tx/tc)
    perf_cu_nx[i].append(tx/tcx)
    gc.collect()
    

    #-- Louvain
    print("\tLouvain  ", end='')
    if i == 0:
        algos.append("Louvain")

    print("n.", end='')
    tx = nx_louvain(pdf)
    print("c.", end='')
    tc = cu_louvain(gdf)
    print("cx.", end='')
    tcx = cu_louvain_nx(pdf)
    print(" ")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    time_algo_cx[i].append(tcx)
    perf[i].append(tx/tc)
    perf_cu_nx[i].append(tx/tcx)
    gc.collect()
    
    #-- TC
    print("\tTC  ", end='')
    if i == 0:
        algos.append("TC")

    print("n.", end='')
    tx = nx_tc(pdf)
    print("c.", end='')
    tc = cu_tc(gdf)
    print("cx.", end='')
    tcx = cu_tc_nx(pdf)
    print(" ")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    time_algo_cx[i].append(tcx)
    perf[i].append(tx/tc)
    perf_cu_nx[i].append(tx/tcx)
    gc.collect()


    #-- WCC
    print("\tWCC  ", end='')
    if i == 0:
        algos.append("WCC")

    print("n.", end='')
    tx = nx_wcc(pdf)
    print("c.", end='')
    tc = cu_wcc(gdf)
    print("cx.", end='')
    tcx = cu_wcc_nx(pdf)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    time_algo_cx[i].append(tcx)
    perf[i].append(tx/tc)
    perf_cu_nx[i].append(tx/tcx)
    gc.collect()
    
    #-- Core Number
    print("\tCore Number  ", end='')
    if i == 0:
        algos.append("Core Number")

    print("n.", end='')
    tx = nx_core_num(pdf)
    print("c.", end='')
    tc = cu_core_num(gdf)
    print("cx.", end='')
    tcx = cu_core_num_nx(pdf)
    print(" ")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    time_algo_cx[i].append(tcx)
    perf[i].append(tx/tc)
    perf_cu_nx[i].append(tx/tcx)
    gc.collect()

    
    #-- PageRank
    print("\tPageRank  ", end='')
    if i == 0:
        algos.append("PageRank")

    print("n.", end='')
    tx = nx_pagerank(pdf)
    print("c.", end='')
    tc = cu_pagerank(gdf)
    print("cx.", end='')
    tcx = cu_pagerank_nx(pdf)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    time_algo_cx[i].append(tcx)
    perf[i].append(tx/tc)
    perf_cu_nx[i].append(tx/tcx)
    gc.collect()
    
    
    #-- Jaccard
    print("\tJaccard  ", end='')
    if i == 0:
        algos.append("Jaccard")

    print("n.", end='')
    tx = nx_jaccard(pdf)
    print("c.", end='')
    tc = cu_jaccard(gdf)
    print("cx.", end='')
    tcx = cu_jaccard_nx(pdf)
    print(" ")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    time_algo_cx[i].append(tcx)
    perf[i].append(tx/tc)
    perf_cu_nx[i].append(tx/tcx)
    gc.collect()
    

    #-- BFS
    print("\tBFS  ", end='')
    if i == 0:
        algos.append("BFS")

    print("n.", end='')
    tx = nx_bfs(pdf)
    print("c.", end='')
    tc = cu_bfs(gdf)
    print("cx.", end='')
    tcx = cu_bfs_nx(pdf)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    time_algo_cx[i].append(tcx)
    perf[i].append(tx/tc)
    perf_cu_nx[i].append(tx/tcx)
    gc.collect()
    
    
    #-- SSSP
    print("\tSSSP  ", end='')
    if i == 0:
        algos.append("SSP")

    print("n.", end='')
    tx = nx_sssp(pdf)
    print("c.", end='')
    tc = cu_sssp(gdf)
    print("cx.", end='')
    tcx = cu_sssp(gdf)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    time_algo_cx[i].append(tcx)
    perf[i].append(tx/tc)
    perf_cu_nx[i].append(tx/tcx)
    gc.collect()

    # increament count
    i = i + 1



Reading ./data/preferentialAttachment.mtx...
	data in gdf 999970 and data in pandas 999970
	Katz  n.c.cx.
	BC k=100  n. k = 100 c.cx. 
	Louvain  n.c.cx. 
	TC  n.c.cx. 
	WCC  n.c.cx. 
	Core Number  n.c.cx. 
	PageRank  n.c.cx. 
	Jaccard  n.c.cx. 
	BFS  n.c.cx. 
	SSSP  n.c.cx. 
Reading ./data/dblp-2010.mtx...
	data in gdf 1615400 and data in pandas 1615400
	Katz  n.c.cx.
	BC k=100  n. k = 100 c.cx. 
	Louvain  n.c.cx. 
	TC  n.c.cx. 
	WCC  n.c.cx. 
	Core Number  n.c.cx. 
	PageRank  n.c.cx. 
	Jaccard  n.c.cx. 
	BFS  n.c.cx. 
	SSSP  n.c.cx. 
Reading ./data/coPapersCiteseer.mtx...
	data in gdf 32073440 and data in pandas 32073440
	Katz  n.c.cx.
	BC k=100  n. k = 100 c.cx. 
	Louvain  n.c.cx. 
	TC  n.c.cx. 
	WCC  n.c.cx. 
	Core Number  n.c.cx. 
	PageRank  n.c.cx. 
	Jaccard  n.c.cx. 
	BFS  n.c.cx. 
	SSSP  n.c.cx. 
Reading ./data/as-Skitter.mtx...
	data in gdf 22190596 and data in pandas 22190596
	Katz  n.c.cx.
	BC k=100  n. k = 100 c.cx. 
	Louvain  n.c.cx. 
	TC  n.c.cx. 
	WCC  n.c.cx. 
	Core Numb

In [20]:
#Print results
print(algos)

for i in range(num_datasets):
    print(f"{names[i]}")
    print(f"{perf[i]}")

['   ', 'Katz', 'BC Estimate fixed', 'Louvain', 'TC', 'WCC', 'Core Number', 'PageRank', 'Jaccard', 'BFS', 'SSP']
preferentialAttachment
[531.5083672005737, 713.2091362850375, 5404.871910284401, 378.8714209575697, 70.72927362986779, 76.7855488460584, 369.8407050216162, 100.5242813104882, 4.378275208783683, 162.25090274331285]
dblp
[263.7293255857766, 1558.1884974132756, 622.282663375948, 142.59215746942343, 88.19197528607465, 116.20873484788576, 460.3603070425029, 121.2753931555862, 38.357311342936576, 186.59598139534884]
coPapersCiteseer
[2270.9739290077137, 2436.608230097977, 1057.486186959562, 2300.0750485569533, 233.7511393615408, 246.97401172398136, 853.279646090366, 112.73241487709193, 233.03646005823012, 267.6295226503849]
as-Skitter
[1037.8761480097223, 7091.970778835749, 1475.2901788933452, 3950.5098941991014, 228.11245124984197, 245.35607830599636, 547.8739313855568, 98.81747467270701, 222.20389619850272, 326.73959437931654]


In [21]:
#Print results
print("\n------------\n")
print(algos)

for i in range(num_datasets):
    print(f"{names[i]}")
    print(f"{perf_cu_nx[i]}")


------------

['   ', 'Katz', 'BC Estimate fixed', 'Louvain', 'TC', 'WCC', 'Core Number', 'PageRank', 'Jaccard', 'BFS', 'SSP']
preferentialAttachment
[4.003601795825807, 93.08063351361315, 204.47865340368915, 2.7097492953036797, 0.8666907616338123, 1.2867491927399868, 1.9752328393020735, 0.1240993236960286, 0.7203119096547798, 179.41122927863145]
dblp
[4.885862057048338, 122.87364779256156, 31.02199520144101, 2.180625966164683, 1.4324289723528034, 1.3579940932619847, 2.348198022642711, 0.11145559203860335, 0.7265996313793216, 202.49957600312277]
coPapersCiteseer
[8.158465907417547, 49.369787236543914, 9.28701234947398, 12.067711445592089, 0.8232363804394461, 1.3752921888176353, 2.8151237928186994, 0.08727548257474192, 0.798480184263566, 322.0841686740297]
as-Skitter
[3.66123817526995, 631.2091494372795, 20.617572936160165, 34.49437773532841, 0.9264442180418646, 4.828647431829589, 2.2476285215684615, 0.12267649645919085, 0.8469181921567667, 431.6171964493335]


## The following section is to rerun portions of the benchamrks if needed

# arrays to capture performance gains
names = []
algos = []

# Two dimension data
time_algo_cu = []       # will be two dimensional
time_algo_nx = []       # will be two dimensional
perf = []

algos.append("   ")

i = 0
for k,v in data.items():
    time_algo_cu.append([])
    time_algo_nx.append([])
    perf.append([])
    
    # Saved the file Name
    names.append(k)

    # read data
    gdf = read_data(v)
    pdf = gdf.to_pandas()
    print(f"\tdata in gdf {len(gdf)} and data in pandas {len(pdf)}")

    # prep
    tmp_g = create_cu_ugraph(gdf)
    deg = tmp_g.degree()
    deg_max = deg['degree'].max()

    alpha = 1 / deg_max
    num_nodes = tmp_g.number_of_vertices()
    
    del tmp_g
    del deg
    
    
    #----- Algorithm order is same as defined at top ----
    
    # testing BC with large k values
    # BC - Estimate
    print("\tBC k = 0.1%  (x 0.001) ", end='')
    if i == 0:
        algos.append("BC Estimate percent")
    
    k = math.ceil(num_nodes * 0.001)
    
    print("n.", end='')
    tx = nx_bc(pdf, k)
    print("c.", end='')
    tc = cu_bc(gdf, k)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf[i].append(tx/tc)
    gc.collect()    
    
    # increament count
    i = i + 1

#Print results
print(algos)

for i in range(num_datasets):
    print(f"{names[i]}")
    print(f"{perf[i]}")

___
Copyright (c) 2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___